# Integraçao dos dados

### Imports

In [44]:
import pandas as pd

## Carregando raw data

### Dados dos municípios

In [45]:
df_municipio = pd.read_csv('data/municipio/municipios_IBGE_ANSI.txt', sep=",", encoding='latin1')
df_municipio.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [46]:
df_populacao = pd.read_csv('data/municipio/populacaoMunicipiosEstimada2011_2021.csv', sep=";", skiprows=4, nrows=5597, encoding='latin1', header=None)
df_populacao.drop(columns=[4 ,6], inplace=True)
df_populacao.columns = ["tipo", "codigo_ibge", "nome", "populacao_2011", "populacao_2021"]
df_populacao.head()

,tipo,codigo_ibge,nome,populacao_2011,populacao_2021
0,UF,11,RondÃ´nia,1576455,1815278
1,MU,1100015,Alta Floresta D'Oeste,24228,22516
2,MU,1100379,Alto Alegre dos Parecis,12825,13268
3,MU,1100403,Alto ParaÃ­so,17444,22258
4,MU,1100346,Alvorada D'Oeste,16625,13807


In [47]:
df_area = pd.read_excel(open('data/municipio/AreaMunicipios_2021.xls', 'rb'), sheet_name='AR_BR_MUN_2021', nrows=5572) 
df_area.head()

,ID,CD_UF,NM_UF,NM_UF_SIGLA,CD_MUN,NM_MUN,AR_MUN_2021
0,1,11,Rondônia,RO,1100015,ALTA FLORESTA D'OESTE,7067.127
1,2,11,Rondônia,RO,1100023,ARIQUEMES,4426.571
2,3,11,Rondônia,RO,1100031,CABIXI,1314.352
3,4,11,Rondônia,RO,1100049,CACOAL,3793.000
4,5,11,Rondônia,RO,1100056,CEREJEIRAS,2783.300


### Dados da Região de Saúde

In [48]:
df_rs_municipio = pd.read_csv('data/regiao_saude_municipio.csv', sep=";", encoding='latin1')
df_rs_municipio.head()

,"ï»¿""uf""",cidade,ibge,regiao,no_regiao
0,AC,AcrelÃ¢ndia,120001,12002,Baixo Acre e Purus
1,AC,Assis Brasil,120005,12001,Alto Acre
2,AC,BrasilÃ©ia,120010,12001,Alto Acre
3,AC,Bujari,120013,12002,Baixo Acre e Purus
4,AC,Capixaba,120017,12002,Baixo Acre e Purus


In [49]:
df_regiao_saude = pd.read_csv('data/tb_regiao_saude.csv', sep=",", encoding='latin1')
df_regiao_saude.head()

,co_regiao_saude,no_regiao_saude,latitude,longitude,limite_geografico
0,25006,6Âª RegiÃ£o,-7.034687,-37.283091,0103000020421200000100000017030000C22861D6E681...
1,35093,MarÃ­lia,-22.271246,-49.880964,01030000204212000001000000C717000072BFF7C21AD1...
2,29015,JequiÃ©,-13.707028,-40.301566,010300002042120000010000001B2C00000D63F7826EE9...
3,31095,Serro,-18.764906,-43.427546,01030000204212000001000000EE2900001ADCD6169EE5...
4,35063,Polo Cuesta,-22.937758,-48.335982,010300002042120000010000007511000053D9B2A7DD13...


### Dados Rede Sentinela

In [50]:
df_serv_sentinela = pd.read_csv('data/servicos_sentinelas.csv', sep=",", encoding='latin1')
df_serv_sentinela.head()

,SERVICO,MUNICIPIO,UF,cod_municipio
0,Hospital das ClÃ­nicas do Acre,Rio Branco,AC,1200401
1,Hospital Infantil Yolanda Costa e Silva -\nHos...,Rio Branco,AC,1200401
2,Hospital Geral de Palmas,Palmas,TO,1721000
3,Hospital Geral de Roraima,Boa Vista,RR,1400100
4,Centrode Medicina Tropical -Cemetron,Porto Velho,RO,1100205


### Dados Estados

In [51]:
df_area_estado = pd.read_excel(open('data/municipio/AreaMunicipios_2021.xls', 'rb'), sheet_name='AR_BR_UF_2021', nrows=27) 
df_area_estado.head()



,ID,CD_UF,NM_UF,NM_UF_SIGLA,AR_UF_2021
0,1,11,Rondônia,RO,237765.347
1,2,12,Acre,AC,164173.431
2,3,13,Amazonas,AM,1559167.878
3,4,14,Roraima,RR,223644.530
4,5,15,Pará,PA,1245870.700


### Dados Fluxo Aéreo

In [52]:
df_fluxo_aereo = pd.read_excel(open('data/fluxo_transporte/LIG_AEREAS_2019-2020_fluxos_od.xlsx', 'rb'), sheet_name='BASE_FLUXO_OD', 
                               dtype = {'ID_LIG': int, 'COD_CID_O': int, 'COD_CID_D': int}) 
df_fluxo_aereo.head()

,ID_LIG,COD_UF_O,UF_O,COD_CID_O,NOME_CID_O,NIVEL_O,CLASS_O,COD_UF_D,UF_D,COD_CID_D,NOME_CID_D,NIVEL_D,CLASS_D,VAR01,VAR02,VAR03,VAR04,VAR05,VAR06
0,-2147483648,35.0,SP,3550308,Arranjo Populacional de São Paulo/SP,1A,Grande Metrópole Nacional,33.0,RJ,3304557,Arranjo Populacional do Rio de Janeiro/RJ,1B,Metrópole Nacional,1027294.0,455330.0,318.20,243.99,59,58
1,-2147483648,33.0,RJ,3304557,Arranjo Populacional do Rio de Janeiro/RJ,1B,Metrópole Nacional,35.0,SP,3550308,Arranjo Populacional de São Paulo/SP,1A,Grande Metrópole Nacional,1026248.0,455700.0,320.06,246.94,65,65
2,-2147483648,43.0,RS,4314902,Arranjo Populacional de Porto Alegre/RS,1C,Metrópole,35.0,SP,3550308,Arranjo Populacional de São Paulo/SP,1A,Grande Metrópole Nacional,626155.0,309355.0,312.93,254.07,98,96
3,-2147483648,35.0,SP,3550308,Arranjo Populacional de São Paulo/SP,1A,Grande Metrópole Nacional,43.0,RS,4314902,Arranjo Populacional de Porto Alegre/RS,1C,Metrópole,626063.0,308367.0,318.53,269.18,101,100
4,-2147483648,35.0,SP,3550308,Arranjo Populacional de São Paulo/SP,1A,Grande Metrópole Nacional,29.0,BA,2927408,Arranjo Populacional de Salvador/BA,1C,Metrópole,518955.0,286796.0,414.05,359.07,138,137


### Dados Fluxo Rodoviário e Hidroviário

In [53]:
df_rod_hid = pd.read_csv('data/fluxo_transporte/lig_rodo_hidro.csv', sep=",", encoding='latin1')
df_rod_hid.head()

,Unnamed: 0,ID,COD_UF_A,UF_A,CODMUNDV_A,NOMEMUN_A,COD_UF_B,UF_B,CODMUNDV_B,NOMEMUN_B,...,VAR05,VAR06,VAR07,VAR08,VAR09,VAR10,VAR11,VAR12,VAR13,VAR14
0,0,13026031304260,13,AM,1302603,Manaus,13,AM,1304260,Uarini,...,7.5,0.0,7.5,-60.023335,-3.134691,-65.155346,-2.983609,0.0,NÃ£o,0.023148
1,1,13021081302603,13,AM,1302108,JapurÃ¡,13,AM,1302603,Manaus,...,3.0,0.0,3.0,-66.996903,-1.880845,-60.023335,-3.134691,3.0,NÃ£o,0.026620
2,2,51038095108402,51,MT,5103809,FigueirÃ³polis D'Oeste,51,MT,5108402,VÃ¡rzea Grande,...,0.0,42.0,42.0,-58.737825,-15.446135,-56.144664,-15.658065,0.0,Sim,0.028000
3,3,13000291302603,13,AM,1300029,AlvarÃ£es,13,AM,1302603,Manaus,...,7.5,0.0,7.5,-64.813535,-3.216584,-60.023335,-3.134691,0.0,NÃ£o,0.031250
4,4,13000291301308,13,AM,1300029,AlvarÃ£es,13,AM,1301308,CodajÃ¡s,...,6.0,0.0,6.0,-64.813535,-3.216584,-62.063430,-3.843046,0.0,NÃ£o,0.036232


## Criando tabelas do banco

### Tabela Município

Atributos:
* codigo_ibge (chave primária) ok
* codigo_uf (chave estrangeira) ok
* co_regiao_saude (chave estrangeira) --> feito merge com 1 dígito a menos
* nome ok
* latitude ok
* longitude ok
* populacao_2011 ok
* populacao_2021 ok
* area ok
* densidade_2021

In [54]:
tb_municipio = df_municipio[["codigo_ibge", "nome", "latitude", "longitude", "codigo_uf"]]
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31
2,5200100,Abadiânia,-16.19700,-48.7057,52
3,3100203,Abaeté,-19.15510,-45.4444,31
4,1500107,Abaetetuba,-1.72183,-48.8788,15


In [55]:
tb_municipio = pd.merge(tb_municipio,df_populacao[['codigo_ibge','populacao_2011', 'populacao_2021']],on='codigo_ibge', how='left')
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439


In [56]:
tb_municipio = tb_municipio.merge(df_area[['CD_MUN','AR_MUN_2021']], how='left', left_on='codigo_ibge', right_on='CD_MUN').drop(columns='CD_MUN')
tb_municipio.rename(columns={"AR_MUN_2021": "area_2021"}, inplace=True)
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021,area_2021
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158,143.357
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022,880.461
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873,1044.555
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263,1817.067
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439,1610.654


In [57]:
tb_municipio['densidade_2021']= tb_municipio['populacao_2021']/tb_municipio['area_2021']
tb_municipio.head()

,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021,area_2021,densidade_2021
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158,143.357,63.882475
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022,880.461,7.975367
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873,1044.555,19.982672
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263,1817.067,12.802500
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439,1610.654,99.611090


In [58]:
#Necessario fazer merge com a coluna com o digito a menos, pois os dados 
#da região de saúde estão faltando 1 dígito
tb_municipio["col_apoio"] = tb_municipio['codigo_ibge'].astype(str).str[:-1].astype(int)
tb_municipio = tb_municipio.merge(df_rs_municipio[['ibge','regiao']], how='left', left_on='col_apoio', right_on='ibge').drop(columns=['ibge', 'col_apoio'])
tb_municipio.rename(columns={"regiao": "co_regiao_saude"}, inplace=True)
tb_municipio.head()


,codigo_ibge,nome,latitude,longitude,codigo_uf,populacao_2011,populacao_2021,area_2021,densidade_2021,co_regiao_saude
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023,9158,143.357,63.882475,52001
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724,7022,880.461,7.975367,31074
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088,20873,1044.555,19.982672,52011
3,3100203,Abaeté,-19.15510,-45.4444,31,22716,23263,1817.067,12.802500,31024
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785,160439,1610.654,99.611090,15011


### Tabela Região de Saúde

Atributos:
* co_regiao_saude (chave primária) ok
* nome ok
* latitude ok
* longitude ok
* limite_geográfico ok

In [59]:
tb_regiao_saude = df_regiao_saude
tb_regiao_saude.head()

,co_regiao_saude,no_regiao_saude,latitude,longitude,limite_geografico
0,25006,6Âª RegiÃ£o,-7.034687,-37.283091,0103000020421200000100000017030000C22861D6E681...
1,35093,MarÃ­lia,-22.271246,-49.880964,01030000204212000001000000C717000072BFF7C21AD1...
2,29015,JequiÃ©,-13.707028,-40.301566,010300002042120000010000001B2C00000D63F7826EE9...
3,31095,Serro,-18.764906,-43.427546,01030000204212000001000000EE2900001ADCD6169EE5...
4,35063,Polo Cuesta,-22.937758,-48.335982,010300002042120000010000007511000053D9B2A7DD13...


### Tabela Serviços Sentinelas

Atributos:
* id (index) (chave primária) ok
* nome_serv ok
* cod_municipio ok

In [60]:
tb_serv_sentinela = pd.DataFrame(df_serv_sentinela[['SERVICO', 'cod_municipio']])
tb_serv_sentinela.rename(columns={"SERVICO": "nome_serv"}, inplace=True)
tb_serv_sentinela.index.rename('id', inplace=True)
# tb_serv_sentinela["id"] = tb_serv_sentinela.index
tb_serv_sentinela.head()

,nome_serv,cod_municipio
id,,
0,Hospital das ClÃ­nicas do Acre,1200401
1,Hospital Infantil Yolanda Costa e Silva -\nHos...,1200401
2,Hospital Geral de Palmas,1721000
3,Hospital Geral de Roraima,1400100
4,Centrode Medicina Tropical -Cemetron,1100205


### Tabela de estado

Atributos:
* cod_uf (chave primária) ok
* nome ok
* uf ok
* area ok
* populacao_2011 ok
* populacao_2021 ok
* densidade_2021 ok

In [61]:
tb_estado = df_populacao[df_populacao.tipo == 'UF'][['codigo_ibge', 'populacao_2011', 'populacao_2021']]
tb_estado.head()

,codigo_ibge,populacao_2011,populacao_2021
0,11,1576455,1815278
53,12,746386,906876
76,13,3538387,4269995
139,14,460165,652713
155,15,7688593,8777124


In [62]:
tb_estado = tb_estado.merge(df_area_estado[['CD_UF','AR_UF_2021', 'NM_UF', 'NM_UF_SIGLA']], how='left', left_on='codigo_ibge', right_on='CD_UF').drop(columns='CD_UF')
tb_estado.rename(columns={"AR_UF_2021": "area_2021", "NM_UF_SIGLA": "uf", "NM_UF":"nome"}, inplace=True)
tb_estado.head()

,codigo_ibge,populacao_2011,populacao_2021,area_2021,nome,uf
0,11,1576455,1815278,237765.347,Rondônia,RO
1,12,746386,906876,164173.431,Acre,AC
2,13,3538387,4269995,1559167.878,Amazonas,AM
3,14,460165,652713,223644.530,Roraima,RR
4,15,7688593,8777124,1245870.700,Pará,PA


In [63]:
tb_estado['densidade_2021'] = tb_estado['populacao_2021']/tb_estado['area_2021']
tb_estado.head()

,codigo_ibge,populacao_2011,populacao_2021,area_2021,nome,uf,densidade_2021
0,11,1576455,1815278,237765.347,Rondônia,RO,7.634746
1,12,746386,906876,164173.431,Acre,AC,5.523890
2,13,3538387,4269995,1559167.878,Amazonas,AM,2.738637
3,14,460165,652713,223644.530,Roraima,RR,2.918529
4,15,7688593,8777124,1245870.700,Pará,PA,7.044972


### Tabela de Fluxo Aéreo

Atributos:
* id ok (chave primária)
* total_passagens_2019 ok
* total_passagens_2020 ok
* tempo_medio_2019 ok
* tempo_medio_2020 ok
* cod_mun_origem ok (chave estrangeira)
* cod_mun_destino ok (chave estrangeira)

In [64]:
tb_fluxo_aereo = pd.DataFrame(df_fluxo_aereo[['ID_LIG', 'COD_CID_O', 'COD_CID_D', 'VAR01', 'VAR02', 'VAR05', 'VAR06']])
tb_fluxo_aereo.rename(columns={"ID_LIG": "id_lig", "COD_CID_O": "cod_mun_origem", "COD_CID_D": "cod_mun_destino", 
                          "VAR01":"total_pass_2019", "VAR02":"total_pass_2020", "VAR05":"tempo_medio_2019",
                          "VAR06":"tempo_medio_2020"}, inplace=True)

tb_fluxo_aereo = tb_fluxo_aereo.fillna(0)
tb_fluxo_aereo['total_pass_2019'] = tb_fluxo_aereo['total_pass_2019'].astype(int)
tb_fluxo_aereo['total_pass_2020'] = tb_fluxo_aereo['total_pass_2020'].astype(int)

tb_fluxo_aereo.head()

,id_lig,cod_mun_origem,cod_mun_destino,total_pass_2019,total_pass_2020,tempo_medio_2019,tempo_medio_2020
0,-2147483648,3550308,3304557,1027294,455330,59,58
1,-2147483648,3304557,3550308,1026248,455700,65,65
2,-2147483648,4314902,3550308,626155,309355,98,96
3,-2147483648,3550308,4314902,626063,308367,101,100
4,-2147483648,3550308,2927408,518955,286796,138,137


### Tabela de Fluxo Rodoviário

Atributos:
* id (chave primária)ok
* cod_mun_A (chave estrangeira) ok
* cod_mun_B (chave estrangeira) ok
* tempo_medio_2016 
* freq_saida_2016 ok


In [65]:
tb_fluxo_rod = pd.DataFrame(df_rod_hid[['ID', 'CODMUNDV_A', 'CODMUNDV_B', 'VAR06']])
tb_fluxo_rod.rename(columns={"ID": "id", "CODMUNDV_A": "cod_mun_A", "CODMUNDV_B": "cod_mun_B", "VAR06": "freq_saida_2016"}, inplace=True)
tb_fluxo_rod.head()


,id,cod_mun_A,cod_mun_B,freq_saida_2016
0,13026031304260,1302603,1304260,0.0
1,13021081302603,1302108,1302603,0.0
2,51038095108402,5103809,5108402,42.0
3,13000291302603,1300029,1302603,0.0
4,13000291301308,1300029,1301308,0.0


### Tabela de Fluxo Hidroviário

Atributos:
* id (chave primária)ok
* cod_mun_A (chave estrangeira) ok
* cod_mun_B (chave estrangeira) ok
* tempo_medio_2016 
* freq_saida_2016 ok

In [66]:
tb_fluxo_hid = pd.DataFrame(df_rod_hid[['ID', 'CODMUNDV_A', 'CODMUNDV_B', 'VAR05']])
tb_fluxo_hid.rename(columns={"ID": "id", "CODMUNDV_A": "cod_mun_A", "CODMUNDV_B": "cod_mun_B", "VAR05": "freq_saida_2016"}, inplace=True)
tb_fluxo_hid.head()

,id,cod_mun_A,cod_mun_B,freq_saida_2016
0,13026031304260,1302603,1304260,7.5
1,13021081302603,1302108,1302603,3.0
2,51038095108402,5103809,5108402,0.0
3,13000291302603,1300029,1302603,7.5
4,13000291301308,1300029,1301308,6.0


## Salvando Tabelas

In [68]:
#Tabela de município
tb_municipio.to_csv("data/integrado/municipio.csv", index=False)

#Tabela de rede sentinela
tb_serv_sentinela.to_csv("data/integrado/serv_sentinela.csv", index=True)

#Tabela de regiao de saúde
tb_regiao_saude.to_csv("data/integrado/regiao_saude.csv", index=False)

#Tabela de estados
tb_estado.to_csv("data/integrado/estado.csv", index=False)

#Tabela de fluxo aéro
tb_fluxo_aereo.to_csv("data/integrado/fluxo_aereo.csv", index=False)

#Tabela de fluxo rodoviário
tb_fluxo_rod.to_csv("data/integrado/fluxo_rod.csv", index=False)

#Tabela de fluxo hidroviário
tb_fluxo_hid.to_csv("data/integrado/fluxo_hidro.csv", index=False)